In [ ]:
from capsnet.fine_tuning_helper import load_study

study_name = "main"
study = load_study(study_name)

In [ ]:
try:
    display(study.best_value)
    display(study.best_params)
except:
    print("Empty Study")

In [ ]:
from capsnet.dataset_helper import load_raw_datasets

train_dataset_raw, val_dataset_raw, test_dataset_raw = load_raw_datasets()

In [ ]:
from capsnet.fine_tuning_helper import objective, save_study
from capsnet.dataset_helper import CacheLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cache_loader = CacheLoader(train_dataset_raw, val_dataset_raw)

study.optimize(
    lambda trial: objective(trial, 15, device, cache_loader),
    n_trials=10
)

save_study(study_name, study)

In [ ]:
display(study.best_value)
display(study.best_params)

In [ ]:
from capsnet.fine_tuning_helper import study_to_model_params
from capsnet.model import CapsNet
from capsnet.train_helper import train_capsnet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_best_params, alpha, lr = study_to_model_params(study.best_params, conv_channels=[64, 128, 256], num_classes=6, capsule_out_dim=16)

train_loader, val_loader = cache_loader.get_or_create_cache(model_best_params["input_size"])

best_model = CapsNet(**model_best_params).to(device)

best_model_trained, top3_f1_avg = train_capsnet(best_model, train_loader, val_loader, device, epochs=20, alpha=alpha, lr=lr, progress=True)

In [ ]:
from capsnet.model import save_model

save_model(best_model_trained, model_best_params, "main")